In [52]:
# import everything we may need
import mysql.connector
from mysql.connector import errorcode
import config
import requests
import time
import json
import pandas as pd
%load_ext autoreload
%autoreload 2
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
#get data from box office mojo as json file
f = open('mojo_data.json')
data = json.load(f)

In [145]:
cnx.rollback()
cursor.close()
cnx.close()
# connect to AWS
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = 'Mod_1_Project',
    buffered = True
)

# Establish cursor
cursor = cnx.cursor()

In [4]:
## function to close the connection
def close_connections():
    cursor.close()
    conn.close()

In [5]:
#name the db
db_name = 'Mod_1_Project'

In [6]:
# create database in AWS
def create_database(cursor, database_name):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database_name))
    except mysql.connector.Error as err:
        ## Catch the error if an error occurs.
        print("Failed creating database: {}".format(err))
        exit(1)
        
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [67]:
#get list of movies from box office mojo data
titles = [x['title'] for x in data]
titles

['The Boss Baby',
 "A Dog's Purpose",
 'Jigsaw',
 'Guardians of the Galaxy Vol. 2',
 'Split',
 'Jumanji:Welcome to the Jungle',
 'All Eyez on Me',
 'Downsizing',
 'Annabelle:Creation',
 'Despicable Me 3',
 'Spider-Man:Homecoming',
 "Molly's Game",
 'The LEGO Batman Movie',
 'American Assassin',
 'Geostorm',
 'Star Wars:The Last Jedi',
 'Transformers:The Last Knight',
 'Baby Driver',
 'Ghost in the Shell (2017)',
 'Atomic Blonde',
 'Beauty and the Beast (2017)',
 'Justice League',
 'Logan Lucky',
 "Tyler Perry's Boo 2! A Madea Halloween",
 'Rings',
 'Victoria and Abdul',
 'Smurfs:The Lost Village',
 'I, Tonya',
 'Baywatch',
 'The Shape of Water',
 'Rough Night',
 'Thor:Ragnarok',
 'Valerian and the City of a Thousand Planets',
 'Alien:Covenant',
 'American Made',
 'Captain Underpants:The First Epic Movie',
 'The Dark Tower',
 'Get Out',
 'Lady Bird',
 'Coco',
 'The Post',
 'John Wick:Chapter Two',
 'The Foreigner',
 'The Fate of the Furious',
 'Leap!',
 'Kidnap (2017)',
 'Pirates of the

In [69]:
key = config.apiKey
url = 'http://www.omdbapi.com/?apikey='

# Make call to API
# Iterate over list of titles (from box office mojo) to retrieve all

def OMDB_call(titles):
    all_titles = []
    for title in titles: 
        parameters = {'t': {title}}
        response = requests.get(url+key, params=parameters)
        data = response.json()
        all_titles.append(data)
        time.sleep(.5)
    return all_titles

movie_dict = OMDB_call(titles[:100])

In [71]:
movie_dict[0]

{'Title': 'The Boss Baby',
 'Year': '2017',
 'Rated': 'PG',
 'Released': '31 Mar 2017',
 'Runtime': '97 min',
 'Genre': 'Animation, Adventure, Comedy, Family, Fantasy',
 'Director': 'Tom McGrath',
 'Writer': 'Michael McCullers, Marla Frazee (based on the book by)',
 'Actors': 'Alec Baldwin, Steve Buscemi, Jimmy Kimmel, Lisa Kudrow',
 'Plot': 'A suit-wearing, briefcase-carrying baby pairs up with his 7-year old brother to stop the dastardly plot of the CEO of Puppy Co.',
 'Language': 'English, Spanish',
 'Country': 'USA',
 'Awards': 'Nominated for 1 Oscar. Another 1 win & 19 nominations.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg5MzUxNzgxNV5BMl5BanBnXkFtZTgwMTM2NzQ3MjI@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.3/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '52%'},
  {'Source': 'Metacritic', 'Value': '50/100'}],
 'Metascore': '50',
 'imdbRating': '6.3',
 'imdbVotes': '94,906',
 'imdbID': 'tt3874544',
 'Type': 'movie',
 'DVD': '25 Jul

In [ ]:
#function to create tables in our database
def create_table(dict_of_tables, db_name):
    for table_name in dict_of_tables:
        table_query = dict_of_tables[table_name]
        cursor.execute("USE {}".format(db_name))
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute(table_query)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("already exists.")
            else:
                print(err.msg)
        else:
            print("OK")


In [46]:
#Create tables to input data
TABLES = {}
TABLES['main_movie_table'] = (
    "CREATE TABLE main_movie_table ("
    "  movie_id int NOT NULL AUTO_INCREMENT,"
    "  title text NOT NULL,"
    "  budget int(10),"
    "  release_date date,"
    "  award_wins int(3),"
    "  award_nomination int(3),"
    "  RT_rating int(2),"
    "  IMDB_rating real,"
    "  PRIMARY KEY (movie_id)"
    ") ENGINE=InnoDB")

TABLES['genres'] = (
    "CREATE TABLE genres ("
    "  genre_id int NOT NULL,"
    "  genre text,"
    "  PRIMARY KEY (genre_id)"
    ") ENGINE=InnoDB")

TABLES['key_words'] = (
    "CREATE TABLE key_words ("
    "  keyword_id int NOT NULL,"
    "  key_words text,"
    "  PRIMARY KEY (keyword_id)"
    ") ENGINE=InnoDB")

TABLES['movies_genres'] = (
    "CREATE TABLE movies_genres ("
      " movie_id int NOT NULL,"
      " genre_id int NOT NULL"
      ") ENGINE=InnoDB")

TABLES['movies_keywords'] = (
    "CREATE TABLE movies_keywords ("
      " movie_id int NOT NULL,"
      " keyword_id text NOT NULL"
      ") ENGINE=InnoDB")

TABLES['actors'] = (
    "CREATE TABLE actors ("
    " actor_id int NOT NULL AUTO_INCREMENT," 
    " actor_name text NOT NULL,"
    " PRIMARY KEY (actor_id)"
    ") ENGINE=InnoDB")

TABLES['movie_actors'] = (
    "CREATE TABLE movie_actors("
    " movie_id int NOT NULL,"
    " actor_id int NOT NULL"
    ") ENGINE=InnoDB")

create_table(TABLES, 'Mod_1_Project')

Creating table main_movie_table: OK
Creating table genres: already exists.
Creating table key_words: already exists.
Creating table movies_genres: already exists.
Creating table movies_keywords: already exists.
Creating table actors: already exists.
Creating table movie_actors: already exists.


In [ ]:
#insert data into tables
def add_movies_toDB():
    for movie in movie_dict:
        ('''insert into main_movie_table (title, release_date, award_wins, award_nomination, 
        RT_rating, IMDB_rating) values(%s,%s,%s,%s,%s,%s)''',
        movie['Title'], movie['Released'], movie['Awards'], 
        movie['Rating'][1]['Value'].replace('%', ''), movie['imdbRating']) 
        conn.commit
         
def add_genres_toDB():
    for movie in movie_dict:
        ('''insert into genres (genre) values(%s)''', movie['Genre'])
        conn.commit
            
## Drop all tables to refresh
def drop_everything():
    cursor.execute("""DROP TABLE Reviews;""")
    cnx.commit()
    cursor.execute("""DROP TABLE Businesses;""")
    cnx.commit()

In [119]:
import sys
def get_movie_id(title):
    title = title.replace(r"'",r"\'")
    cursor.execute(f"""SELECT movie_id FROM main_movie_table WHERE bom_title = '{title}' OR omdb_title = '{title}';""")
    return cursor.fetchone()

def get_actor_id(actor_name):
    cursor.execute(f"""SELECT actor_id FROM actors WHERE actor_name = '{actor_name}';""")
    return cursor.fetchone()

def add_actors_toDB(omdb_data):
    idx = 0
    sys.stdout.write("INSERTING ACTORS")
    sys.stdout.flush()
    for movie in omdb_data.itertuples():
        try:
            if movie.Genre != 'Documentary':
                if ',' in movie.Actors:
                    for actor in movie.Actors.split(', '):
                        actor = actor.replace(r"'",r"\'")
                        #get actor_id if already present in database
                        actor_id = get_actor_id(actor)
                        #if actor not present, insert and get id back
                        if actor_id is None:
                            actor_insert_q = f"""INSERT INTO actors(actor_name) VALUES('{actor}');"""
                            cursor.execute(actor_insert_q)
                            cursor.execute("""SELECT LAST_INSERT_ID();""")
                            actor_id = cursor.fetchone()[0]
                        else:
                            actor_id = actor_id[0]

                        #get id of movie
                        movie_id = get_movie_id(movie.Title)[0]

                        insert_actor_movie_q = f"""INSERT INTO movie_actors(movie_id, actor_id) VALUES ('{movie_id}', '{actor_id}');"""
                        #add actor + movie to movie_actors
                        cursor.execute(insert_actor_movie_q)
                        idx += 1
                        if idx % 112 == 0:
                            sys.stdout.write(".")
                            sys.stdout.flush()
                else:
                    print(movie)
            else:
                print(movie)
        except TypeError:
            print('Non-iterable actors movie', movie)
    #add_actors_toDB()  
    cnx.commit()
    sys.stdout.write("OK")
    sys.stdout.flush()
omdb_data = pd.read_csv('omdb_data.csv')
add_actors_toDB(omdb_data)

INSERTING ACTORS...Pandas(Index=97, _1=97, Actors='Al Gore, Cory Booker, George W. Bush, Bill Clinton', Awards='Nominated for 1 BAFTA Film Award. Another 3 wins & 12 nominations.', BoxOffice='$3,456,144', Country='USA', DVD='24 Oct 2017', Director='Bonni Cohen, Jon Shenk', Genre='Documentary', Language='English, French, Russian, German, Mandarin', Metascore=68.0, Plot='A decade after An Inconvenient Truth (2006) brought climate change to the heart of popular culture, the follow-up shows just how close we are to a real energy revolution.', Poster='https://m.media-amazon.com/images/M/MV5BNTkzNTQ0MzA3Ml5BMl5BanBnXkFtZTgwNzg4OTcyMjI@._V1_SX300.jpg', Production='Paramount Pictures', Rated='PG', Ratings="[{'Source': 'Internet Movie Database', 'Value': '6.3/10'}, {'Source': 'Rotten Tomatoes', 'Value': '79%'}, {'Source': 'Metacritic', 'Value': '68/100'}]", Released='04 Aug 2017', Response=True, Runtime='98 min', Title='An Inconvenient Sequel: Truth to Power', Type='movie', Website='https://inc

.Pandas(Index=217, _1=217, Actors='Lee Hwa-Choon, Moon Jae-in, Roh Moo-Hyun, Rhyu Si-Min', Awards=nan, BoxOffice=nan, Country='South Korea', DVD=nan, Director='Chang-jae Lee', Genre='Documentary', Language='English', Metascore=nan, Plot="A look at the life of Roh Moo-hyun, South Korea's president from 2003-2008.", Poster='https://images-na.ssl-images-amazon.com/images/M/MV5BMTc0ODM2OTgyNF5BMl5BanBnXkFtZTgwMzg3ODc2MjI@._V1_SX300.jpg', Production='CJ Entertainment', Rated=nan, Ratings="[{'Source': 'Internet Movie Database', 'Value': '8.4/10'}]", Released='02 Jun 2017', Response=True, Runtime='119 min', Title='Our President', Type='movie', Website=nan, Writer=nan, Year='2017', bom_title='Our President', imdbID='tt7037566', imdbRating=8.4, imdbVotes='34', totalSeasons=nan)
Pandas(Index=221, _1=221, Actors='Agnès Varda, JR, Jeannine Carpentier, Clemens Van Dungern', Awards='Nominated for 1 Oscar. Another 32 wins & 31 nominations.', BoxOffice='$900,903', Country='France', DVD='06 Mar 2018', 

Pandas(Index=304, _1=304, Actors='Elsa Dorfman', Awards=nan, BoxOffice='$121,498', Country='USA', DVD=nan, Director='Errol Morris', Genre='Documentary', Language='English', Metascore=77.0, Plot='A look at the life and work of photographer Elsa Dorfman.', Poster='https://m.media-amazon.com/images/M/MV5BY2EzYjQ3YmQtN2YyYi00YjUyLWI0MTctYTRkNmNjYmRiOTI1XkEyXkFqcGdeQXVyMjU3NTI0Mg@@._V1_SX300.jpg', Production='NEON', Rated='R', Ratings="[{'Source': 'Internet Movie Database', 'Value': '6.6/10'}, {'Source': 'Rotten Tomatoes', 'Value': '96%'}, {'Source': 'Metacritic', 'Value': '77/100'}]", Released='30 Jun 2017', Response=True, Runtime='76 min', Title="The B-Side: Elsa Dorfman's Portrait Photography", Type='movie', Website='http://bsidefilm.com/', Writer=nan, Year='2016', bom_title="The B-Side:Elsa Dorfman's Portrait Photography", imdbID='tt5952468', imdbRating=6.6, imdbVotes='395', totalSeasons=nan)
Pandas(Index=305, _1=305, Actors='Thomas Sung, Hwei Lin Sung, Cyrus Vance Jr., Matt Taibbi', Aw

Pandas(Index=359, _1=359, Actors='Tony Benatatos, Jamal Braithwaite, Steve Buscemi, Joe Casaliggi', Awards='Won 2 Primetime Emmys. Another 6 wins & 6 nominations.', BoxOffice=nan, Country='USA, France', DVD='12 Sep 2002', Director='James Hanlon, Gédéon Naudet, Jules Naudet', Genre='Documentary', Language='English', Metascore=nan, Plot="A real life documentary following the events of September 11 from an insider's view, through the lens of James Hanlon and two French filmmakers who were in Manhattan that one day.", Poster='https://m.media-amazon.com/images/M/MV5BMTIyMzUyNzYzNV5BMl5BanBnXkFtZTcwMTk2NzEyMQ@@._V1_SX300.jpg', Production='Goldfish Pictures', Rated='TV-PG', Ratings="[{'Source': 'Internet Movie Database', 'Value': '8.4/10'}]", Released='10 Mar 2002', Response=True, Runtime='112 min', Title='9/11', Type='movie', Website=nan, Writer='Tom Forman, Greg Kandra', Year='2002', bom_title='9/11', imdbID='tt0312318', imdbRating=8.4, imdbVotes='6,032', totalSeasons=nan)
.Non-iterable act

Pandas(Index=396, _1=396, Actors=nan, Awards=nan, BoxOffice=nan, Country='USA, Australia, Canada, India, UK', DVD=nan, Director='Laurel Chiten', Genre='Documentary', Language='English', Metascore=nan, Plot='A film about the most controversial form of medicine ever invented. Homeopathy', Poster='https://images-na.ssl-images-amazon.com/images/M/MV5BZDdiMzEzNjgtYjQyZS00NDU4LTk3NjgtOWMxYTBjMTA2Njc4XkEyXkFqcGdeQXVyNDk2NzI4OQ@@._V1_SX300.jpg', Production=nan, Rated=nan, Ratings='[]', Released='01 Apr 2017', Response=True, Runtime='66 min', Title='Just One Drop', Type='movie', Website=nan, Writer=nan, Year='2017', bom_title='Just One Drop', imdbID='tt6874242', imdbRating=nan, imdbVotes=nan, totalSeasons=nan)
Pandas(Index=397, _1=397, Actors='Dina Buno, Scott Levin, Frank Costanzo, Jeannie Levin', Awards='6 wins & 3 nominations.', BoxOffice='$90,873', Country='USA', DVD=nan, Director='Antonio Santini, Dan Sickles', Genre='Documentary', Language='English', Metascore=75.0, Plot='An eccentric sub

...

Pandas(Index=1452, _1=1452, Actors='Jex Blackmore, Chalice Blythe, Nicholas Crowe, Sal De Ciccio', Awards=nan, BoxOffice=nan, Country='USA', DVD=nan, Director='Penny Lane', Genre='Documentary', Language='English', Metascore=76.0, Plot='A look at the quick rise and influence of the controversial religious group known as The Satanic Temple.', Poster='https://m.media-amazon.com/images/M/MV5BMjRlNGY2MTUtYzNmZi00ZDRjLWFjNWQtOWJjZDZjMDhmMDAyXkEyXkFqcGdeQXVyMzQyMzAxNjg@._V1_SX300.jpg', Production='Magnolia Pictures', Rated='R', Ratings="[{'Source': 'Internet Movie Database', 'Value': '6.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '97%'}, {'Source': 'Metacritic', 'Value': '76/100'}]", Released='17 Apr 2019', Response=True, Runtime='95 min', Title='Hail Satan?', Type='movie', Website='https://www.hailsatanfilm.com/', Writer=nan, Year='2019', bom_title='Hail Satan?', imdbID='tt9358044', imdbRating=6.8, imdbVotes='663', totalSeasons=nan)
Pandas(Index=1454, _1=1454, Actors='John Lydon, Martin A

Pandas(Index=1496, _1=1496, Actors='Dante Barksdale, Monique Brown, Clayton Gunton, Tia Latrell', Awards=nan, BoxOffice=nan, Country='USA', DVD=nan, Director='Marilyn Ness', Genre='Documentary', Language='English', Metascore=85.0, Plot='During three years of unparalleled violence in Baltimore, CHARM CITY delivers an unexpectedly candid, observational portrait of those left on the frontlines. With grit, fury, and compassion...', Poster='https://m.media-amazon.com/images/M/MV5BNGViYjBiYTAtMTA3Mi00ZGMxLWI0NjEtZGRjNTE3OWEwMWE3XkEyXkFqcGdeQXVyMzAyMzE2MzM@._V1_SX300.jpg', Production=nan, Rated='TV-MA', Ratings="[{'Source': 'Internet Movie Database', 'Value': '7.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '100%'}, {'Source': 'Metacritic', 'Value': '85/100'}]", Released='22 Apr 2018', Response=True, Runtime='108 min', Title='Charm City', Type='movie', Website=nan, Writer=nan, Year='2018', bom_title='Charm City', imdbID='tt8196200', imdbRating=7.4, imdbVotes='98', totalSeasons=nan)
Non-itera

Pandas(Index=1574, _1=1574, Actors='Travis Wilkerson', Awards='1 nomination.', BoxOffice=nan, Country='USA', DVD=nan, Director='Travis Wilkerson', Genre='Documentary', Language='English', Metascore=78.0, Plot="A documentary murder mystery about the filmmaker's family, set in lower Alabama.", Poster='https://m.media-amazon.com/images/M/MV5BNTY0Njc1NjQzMl5BMl5BanBnXkFtZTgwMjg4OTQ3NDM@._V1_SX300.jpg', Production='Grasshopper Film', Rated=nan, Ratings="[{'Source': 'Internet Movie Database', 'Value': '6.7/10'}, {'Source': 'Rotten Tomatoes', 'Value': '84%'}, {'Source': 'Metacritic', 'Value': '78/100'}]", Released='20 Jan 2017', Response=True, Runtime='90 min', Title='Did You Wonder Who Fired the Gun?', Type='movie', Website='http://grasshopperfilm.com/film/did-you-wonder-who-fired-the-gun/', Writer='Travis Wilkerson (story)', Year='2017', bom_title='Did You Wonder Who Fired the Gun?', imdbID='tt6370614', imdbRating=6.7, imdbVotes='190', totalSeasons=nan)
Pandas(Index=1579, _1=1579, Actors='N

.Pandas(Index=1628, _1=1628, Actors='Georgia Angelopoulos, Jacques Berchtold, J. Keith Elliott, Craig A. Evans', Awards=nan, BoxOffice=nan, Country='USA', DVD=nan, Director='Reuben Evans', Genre='Documentary', Language='English', Metascore=nan, Plot='Dr. Craig Evans travels the globe to track down the most ancient New Testament manuscripts. Along the way, he highlights groundbreaking new evidence, demonstrating the reliability of the New Testament manuscripts.', Poster='https://m.media-amazon.com/images/M/MV5BNTZmODg2MjgtNjRlYi00N2YyLWFhMTYtZDE1YTU0MDg5MWNmXkEyXkFqcGdeQXVyODAxNzk2NjM@._V1_SX300.jpg', Production='Fathom Events', Rated=nan, Ratings="[{'Source': 'Internet Movie Database', 'Value': '8.9/10'}]", Released='24 Apr 2018', Response=True, Runtime='75 min', Title='Fragments of Truth', Type='movie', Website=nan, Writer='Reuben Evans', Year='2018', bom_title='Fragments of Truth', imdbID='tt7499368', imdbRating=8.9, imdbVotes='16', totalSeasons=nan)
Pandas(Index=1632, _1=1632, Actor

Pandas(Index=1675, _1=1675, Actors=nan, Awards=nan, BoxOffice=nan, Country='USA', DVD=nan, Director='Jeremy Frindel', Genre='Documentary', Language='English, Marathi', Metascore=nan, Plot="The story of Dr. Vasant Lad's mission to bring the ancient health care system of wellness called Ayurveda from India to the West in the late 1970s. From the director of One Track Heart: The Story of Krishna Das.", Poster='https://ia.media-imdb.com/images/M/MV5BMGI2ODA2NDUtN2IxNi00NjEwLWFkOTItNDA0ZmIyZTA0MDVkXkEyXkFqcGdeQXVyMjI3NDAyNg@@._V1_SX300.jpg', Production='Zeitgeist Films', Rated='UNRATED', Ratings="[{'Source': 'Internet Movie Database', 'Value': '6.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '40%'}]", Released='02 Feb 2018', Response=True, Runtime='89 min', Title='The Doctor from India', Type='movie', Website=nan, Writer=nan, Year='2018', bom_title='The Doctor from India', imdbID='tt7999770', imdbRating=6.8, imdbVotes='11', totalSeasons=nan)
Pandas(Index=1682, _1=1682, Actors='Willem Dafoe

In [11]:
#insert keywords into database
movie_keywords = pd.read_csv('improved_import_movies.csv')
keywords = movie_keywords['keyword'].unique()
cursor.execute("""USE Mod_1_Project;""")
for keyword in keywords:
    insert_keyword_q = f"""INSERT INTO key_words(key_words) VALUES ('{keyword}');"""
    cursor.execute(insert_keyword_q)
    cnx.commit()

In [118]:
from urllib.parse import urlencode
from imdb import IMDb
import sys

idx = 0
sys.stdout.write('[')
sys.stdout.flush()
ia = IMDb()
s = requests.Session()
movie_keywords = pd.read_csv('improved_import_movies.csv')
movie_keywords = movie_keywords[movie_keywords['release_date'] < '2017-01-01']
titles = movie_keywords['title']
expanded_movie_data = []
url_stem = 'http://www.omdbapi.com/?apikey=' + apiKey + '&'

for title in titles:
    url_title = urlencode({'t':title})
    omdb_data = json.loads(s.get(url_stem+url_title).text)
    time.sleep(1)
    try:
        win = list(get_awards(title,ia).keys())[0]
        nom = list(get_awards(title,ia).keys())[1]
        award_wins = get_awards(title,ia)[win]
        award_noms = get_awards(title,ia)[nom]
    except:
        award_wins = 'NULL'
        award_noms = 'NULL'
    expanded_movie_data.append({
        'title': title.replace(r"'", r"\'"),
        'award_wins': award_wins,
        'award_nomination': award_noms,
        'RT_rating': [rating['Value'][:-1] for rating in omdb_data['Ratings'] if rating['Source'] == 'Rotten Tomatoes'],
        'IMDB_rating': [rating['Value'][:3] for rating in omdb_data['Ratings'] if rating['Source'] == 'Internet Movie Database']
    })
    idx += 1
    if idx % 17 == 0:
        sys.stdout.write('=')
        sys.stdout.flush()
sys.stdout.write(']')
sys.stdout.flush()
expanded_movie_data
# "  title text NOT NULL,"
#     "  budget int(10),"
#     "  release_date date,"
#     "  award_wins int(3),"
#     "  award_nomination int(3),"
#     "  RT_rating int(2),"
#     "  IMDB_rating real,"

[=======================================

2019-07-31 15:27:31,396 CRITICAL [imdbpy] /Users/brendan.gilroy/anaconda3/envs/learn-env/lib/python3.7/site-packages/imdb/_exceptions.py:34: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/find?q=Reservoir+Dogs&s=tt', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/Users/brendan.gilroy/anaconda3/envs/learn-env/lib/python3.7/site-packages/imdb/parser/http/__init__.py", line 220, in retrieve_unicode
    content = response.read()
  File "/Users/brendan.gilroy/anaconda3/envs/learn-env/lib/python3.7/http/client.py", line 454, in read
    return self._readall_chunked()
  File "/Users/brendan.gilroy/anaconda3/envs/learn-env/lib/python3.7/http/client.py", line 564, in _readall_chunked
    value.append(self._safe_read(chunk_left))
  File "/Users/brendan.gilroy/anaconda3/envs/learn-env/lib/python3.7/http/client.py", line 61

===============]

[{'title': 'Forrest Gump',
  'award_wins': 47,
  'award_nomination': 68,
  'RT_rating': ['72'],
  'IMDB_rating': ['8.8']},
 {'title': 'A League of Their Own',
  'award_wins': 6,
  'award_nomination': 13,
  'RT_rating': ['78'],
  'IMDB_rating': ['7.2']},
 {'title': 'Fargo',
  'award_wins': 59,
  'award_nomination': 203,
  'RT_rating': ['93'],
  'IMDB_rating': ['8.1']},
 {'title': 'There Will Be Blood',
  'award_wins': 109,
  'award_nomination': 136,
  'RT_rating': ['91'],
  'IMDB_rating': ['8.2']},
 {'title': 'Blue Velvet',
  'award_wins': 18,
  'award_nomination': 18,
  'RT_rating': ['94'],
  'IMDB_rating': ['7.8']},
 {'title': 'Walking Tall',
  'award_wins': 'NULL',
  'award_nomination': 'NULL',
  'RT_rating': ['26'],
  'IMDB_rating': ['6.3']},
 {'title': 'W.',
  'award_wins': 1,
  'award_nomination': 7,
  'RT_rating': ['58'],
  'IMDB_rating': ['6.3']},
 {'title': 'Serial Mom',
  'award_wins': 0,
  'award_nomination': 1,
  'RT_rating': ['63'],
  'IMDB_rating': ['6.7']},
 {'title': 'Di

In [18]:
apiKey = config.apiKey
for title in titles:
    url_title = urlencode({'t':title})
    url_stem = 'http://www.omdbapi.com/?apikey='+apiKey+'&'
    omdb_data = json.loads(requests.get(url_stem+url_title).text)
    print(omdb_data['Awards'])

Won 3 Oscars. Another 51 wins & 197 nominations.
Won 1 Oscar. Another 122 wins & 181 nominations.


In [140]:
from bs4 import BeautifulSoup
from imdb import IMDb
import inspect

def get_awards(movie_title, ia):
    try:
        movie = ia.search_movie(movie_title)[0]
    except IndexError:
        return None
    url_stem = 'https://www.imdb.com/title/tt'
    soup = BeautifulSoup(requests.get(url_stem+movie.getID()+'/awards').text)
    try:
        awards_text = soup.select('#main .desc')[0].string
    except IndexError:
        return None
    awards_words = awards_text.split()
    try:
        return {awards_words[3]:int(awards_words[2]), awards_words[6]:int(awards_words[5])}
    except IndexError:
        return None
ia = IMDb()
# for title in titles:
#     print(get_awards(title, ia))

In [73]:
titles

['The Boss Baby',
 "A Dog's Purpose",
 'Jigsaw',
 'Guardians of the Galaxy Vol. 2',
 'Split',
 'Jumanji:Welcome to the Jungle',
 'All Eyez on Me',
 'Downsizing',
 'Annabelle:Creation',
 'Despicable Me 3',
 'Spider-Man:Homecoming',
 "Molly's Game",
 'The LEGO Batman Movie',
 'American Assassin',
 'Geostorm',
 'Star Wars:The Last Jedi',
 'Transformers:The Last Knight',
 'Baby Driver',
 'Ghost in the Shell (2017)',
 'Atomic Blonde',
 'Beauty and the Beast (2017)',
 'Justice League',
 'Logan Lucky',
 "Tyler Perry's Boo 2! A Madea Halloween",
 'Rings',
 'Victoria and Abdul',
 'Smurfs:The Lost Village',
 'I, Tonya',
 'Baywatch',
 'The Shape of Water',
 'Rough Night',
 'Thor:Ragnarok',
 'Valerian and the City of a Thousand Planets',
 'Alien:Covenant',
 'American Made',
 'Captain Underpants:The First Epic Movie',
 'The Dark Tower',
 'Get Out',
 'Lady Bird',
 'Coco',
 'The Post',
 'John Wick:Chapter Two',
 'The Foreigner',
 'The Fate of the Furious',
 'Leap!',
 'Kidnap (2017)',
 'Pirates of the

In [82]:
import re

def find_year_in_title(title):
    year = re.findall('(201\d)', title)
    print(type(year))
    if year:
        return year[0], title.replace("("+year[0]+")", "")
find_year_in_title(titles[17])

<class 'list'>


In [136]:
# import math

# df = pd.DataFrame(expanded_movie_data)
# df['IMDB_rating'] = df['IMDB_rating'].apply(lambda x: x[0] if len(x) > 0 else math.nan )
# df['RT_rating'] = df['RT_rating'].apply(lambda x: x[0] if len(x) > 0 else math.nan)
# df

,IMDB_rating,RT_rating,award_nomination,award_wins,title
0,8.8,72,68,47,Forrest Gump
1,7.2,78,13,6,A League of Their Own
2,8.1,93,203,59,Fargo
3,8.2,91,136,109,There Will Be Blood
4,7.8,94,18,18,Blue Velvet
5,6.3,26,NULL,NULL,Walking Tall
6,6.3,58,7,1,W.
7,6.7,63,1,0,Serial Mom
8,6.0,44,7,1,Divine Secrets of the Ya-Ya Sisterhood
9,7.1,73,4,7,My Dog Skip


In [39]:
df2 = pd.read_csv('improved_import_movies.csv')
df2

,title,keyword,gross,release_date,corrected_budget
0,Forrest Gump,americana,329694499,1994-07-06,55000000.0
1,A League of Their Own,americana,107533928,1992-07-01,40000000.0
2,Fargo,americana,24611975,1996-03-08,7000000.0
3,There Will Be Blood,americana,40222514,2007-12-26,25000000.0
4,Blue Velvet,americana,8551228,1986-09-19,6000000.0
5,Walking Tall,americana,46437717,2004-04-02,46000000.0
6,W.,americana,25534493,2008-10-17,25100000.0
7,Serial Mom,americana,7820688,1994-04-13,13000000.0
8,Divine Secrets of the Ya-Ya Sisterhood,americana,69599016,2002-06-07,27000000.0
9,My Dog Skip,americana,34134641,2000-01-14,6000000.0


In [36]:
# df_keyword_movies = df_keyword_movies.drop(columns=[column for column in df_keyword_movies.columns if '_x' in column])
# df_keyword_movies = df_keyword_movies.rename(columns = {'keyword_y':'keyword',	'gross_y':'gross',	'release_date_y': 'release_date',	'corrected_budget_y':'corrected_budget'})


In [44]:
df_keyword_movies = pd.read_csv('improved_import_movies2.csv')
df_keyword_movies = df_keyword_movies.drop(columns=['Unnamed: 0'])
df_keyword_movies = df_keyword_movies.dropna()
df_keyword_movies

,IMDB_rating,RT_rating,award_nomination,award_wins,title,keyword,gross,release_date,corrected_budget
0,8.8,72.0,68.0,47.0,Forrest Gump,americana,329694499.0,1994-07-06,55000000.0
1,8.8,72.0,68.0,47.0,Forrest Gump,battle,329694499.0,1994-07-06,55000000.0
2,8.8,72.0,68.0,47.0,Forrest Gump,blockbuster,329694499.0,1994-07-06,55000000.0
3,7.2,78.0,13.0,6.0,A League of Their Own,americana,107533928.0,1992-07-01,40000000.0
4,7.2,78.0,13.0,6.0,A League of Their Own,blockbuster,107533928.0,1992-07-01,40000000.0
5,8.1,93.0,203.0,59.0,Fargo,americana,24611975.0,1996-03-08,7000000.0
6,8.2,91.0,136.0,109.0,There Will Be Blood,americana,40222514.0,2007-12-26,25000000.0
7,8.2,91.0,136.0,109.0,There Will Be Blood,business,40222514.0,2007-12-26,25000000.0
8,8.2,91.0,136.0,109.0,There Will Be Blood,competition,40222514.0,2007-12-26,25000000.0
9,7.8,94.0,18.0,18.0,Blue Velvet,americana,8551228.0,1986-09-19,6000000.0


In [87]:
def get_keyword_id(keyword):
    """if keyword present in database, return id, else return None"""
    cursor.execute(f"""SELECT keyword_id FROM key_words WHERE key_words = '{keyword}';""")
    return cursor.fetchone()

for movie_keyword in df_keyword_movies.itertuples():
    movie_id = get_movie_id(movie_keyword.title)
    #if movie not in database, add movie to database
    if movie_id is None:
        sql_keyword_movie_insert_q = f"""INSERT INTO main_movie_table(bom_title, budget, box_office, release_date, award_wins, award_nominations, RT_rating, IMDB_rating) 
                 VALUES ('{movie_keyword.title}', {movie_keyword.corrected_budget}, {movie_keyword.gross}, '{movie_keyword.release_date}', {movie_keyword.award_wins}, {movie_keyword.award_nomination}, {movie_keyword.RT_rating}, {movie_keyword.IMDB_rating});"""    
        cursor.execute(sql_keyword_movie_insert_q)
        cursor.execute("""SELECT LAST_INSERT_ID();""")
        movie_id = cursor.fetchone()[0]
    else:
        movie_id = movie_id[0]
    keyword_id = get_keyword_id(movie_keyword.keyword)
    #if keyword not in database, add keyword to database
    if keyword_id is None:
        keyword_insert_q = f"""INSERT INTO key_words(key_words) VALUES ('{movie_keyword.keyword}');"""
        cursor.execute(keyword_insert_q)
        cursor.execute("""SELECT LAST_INSERT_ID();""")
        keyword_id = cursor.fetchone()[0]
    else:
        keyword_id = keyword_id[0]
    #add row to movies_keywords regardless
    cursor.execute(f"""SELECT 1 FROM movies_keywords WHERE movie_id = '{movie_id}' AND keyword_id = '{keyword_id}'""")
    movie_keyword_row_exists = cursor.fetchone()
    if movie_keyword_row_exists is None:
        movie_keyword_join_table_insert_q = f"""INSERT INTO movies_keywords(movie_id, keyword_id) VALUES ('{movie_id}', '{keyword_id}')"""
        cursor.execute(movie_keyword_join_table_insert_q)

cnx.commit()
    

In [86]:
get_keyword_id('bikered')

In [88]:
# cursor.execute("""SELECT id FROM information_schema.processlist WHERE user = 'brendan';""")

In [90]:
pd.DataFrame(df_keyword_movies['title'].unique())

414

In [98]:
dataframe_title_keywords = df_keyword_movies[['title', 'keyword']].apply(lambda x: ''.join(x), axis=1).unique()

In [101]:
len(dataframe_title_keywords)

833

In [102]:
cursor.execute("""SELECT CONCAT(mmt.bom_title, k.key_words) FROM main_movie_table mmt 
                JOIN movies_keywords mk ON mmt.movie_id = mk.movie_id 
                JOIN key_words k ON mk.keyword_id = k.keyword_id;""")

In [103]:
title_keywords = cursor.fetchall()


In [104]:
sql_title_keywords = [title_keyword[0] for title_keyword in title_keywords]

In [110]:
len(set(dataframe_title_keywords) - set(sql_title_keywords) )

778

In [109]:
movie_id = get_movie_id('Forrest Gump')[0]
keyword_id = get_keyword_id('Americanant')
cursor.execute(f"""SELECT 1 FROM movies_keywords WHERE movie_id = '{movie_id}' AND keyword_id = '{keyword_id}'""")
movie_keyword_row_exists = cursor.fetchone()
movie_keyword_row_exists is None

True

In [152]:
from urllib.parse import urlencode
from imdb import IMDb
import sys

cnx = mysql.connector.connect(
                host = config.host,
                user = config.user,
                passwd = config.password,
                database = 'Mod_1_Project',
                buffered = True
            )

cursor = cnx.cursor()

idx = 0
sys.stdout.write('[')
sys.stdout.flush()
ia = IMDb()
cursor.execute("""SELECT bom_title FROM main_movie_table WHERE award_nominations IS NULL;""")
titles = [title[0] for title in cursor.fetchall()]
try:
    for title in titles:
        
        try:
            win = list(get_awards(title,ia).keys())[0]
            nom = list(get_awards(title,ia).keys())[1]
            award_wins = get_awards(title,ia)[win]
            award_noms = get_awards(title,ia)[nom]
        except Exception as e:
            award_wins = 'NULL'
            award_noms = 'NULL'
        title = title.replace(r"'", r"\'")
        award_update_q = f"""UPDATE main_movie_table 
                            SET award_wins = {award_wins}, award_nominations = {award_noms}
                            WHERE bom_title = '{title}';"""
        cursor.execute(award_update_q)
        idx += 1
        if idx % 23 == 0:
            cnx.commit()
            cursor.close()
            cnx.close()
            cnx = mysql.connector.connect(
                host = config.host,
                user = config.user,
                passwd = config.password,
                database = 'Mod_1_Project',
                buffered = True
            )

            cursor = cnx.cursor()
            sys.stdout.write('=')
            sys.stdout.flush()
finally:
    cnx.commit()
    sys.stdout.write(']')
    sys.stdout.flush()
# "  title text NOT NULL,"
#     "  budget int(10),"
#     "  release_date date,"
#     "  award_wins int(3),"
#     "  award_nomination int(3),"
#     "  RT_rating int(2),"
#     "  IMDB_rating real,"

[=============]

KeyboardInterrupt: 

In [133]:
cursor.execute("""SELECT COUNT(movie_id) FROM main_movie_table WHERE award_nominations IS NULL;""")
cursor.fetchall()

InternalError: Unread result found

In [ ]:
 award_update_q = f"""UPDATE main_movie_table 
                            SET award_wins = {award_wins}, award_nominations = {award_noms}
                            WHERE bom_title = 'Underworld:Blood Wars';"""
        cursor.execute(award_update_q)

In [154]:
win = list(get_awards('Underworld:Blood Wars',ia).keys())[0]
nom = list(get_awards('Underworld:Blood Wars',ia).keys())[1]
award_wins = get_awards('Underworld:Blood Wars',ia)[win]
award_noms = get_awards('Underworld:Blood Wars',ia)[nom]

AttributeError: 'NoneType' object has no attribute 'keys'